# Regresión

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor

In [2]:
path = "C:/Users/franc/Documents/Lic. IA/Semestre 4/Machine learning" # Cambiar por la ruta de los datos
df = pd.read_csv(path + '/data/chess_games_clean2.csv')
df

,Event,White,Black,Result,Hour,WhiteElo,BlackElo,Opening,TimeControl,Termination,...,581,582,583,584,585,586,587,588,589,590
0,1,123,62,2,22,1512,1570,28072,1074584,1,...,163,328,75,137,82,77,163,328,81,137
1,1,457,66,2,22,1465,1436,595,1074584,1,...,270,462,110,462,311,116,739,169,128,99
2,0,44,216,2,22,1665,1765,34752,1074584,1,...,94,183,320,649,181,350,166,172,376,677
3,2,35,1377,2,22,1595,1673,11136,47338,1,...,333,81,105,279,84,101,333,88,105,279
4,4,44,187,2,22,1511,1621,48705,105090,1,...,479,81,243,90,84,191,360,88,243,452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716284,0,121,108,1,16,1311,1531,12341,1074584,1,...,368,19,753,109,20,433,675,21,1,1
716285,2,350,561,1,13,1932,1807,7614,912414,1,...,506,1531,13,33,129,14,96,151,16,659
716286,5,352,50,1,19,1958,1777,62971,579928,0,...,0,0,0,0,0,0,0,0,2,15
716287,1,227,161,1,6,2139,1722,28072,948246,3,...,122,227,84,334,67,88,300,480,95,431


In [3]:
df = df.select_dtypes(include=['float64', 'int64'])

In [4]:
# Separanado las variables dependientes e independientes
X = df.drop(columns='WhiteElo')
y = df['WhiteElo']  

# Estandarizando los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

## Regresión lineal

In [18]:
# creando un pipeline antes de la regresión lasso
pipeline = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=0.90)), ('linealregression', LinearRegression())])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [19]:
#Evaluando el modelo
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 29161.53
Coefficient of determination: 0.59


## Regresión Lasso

In [15]:
# creando un pipeline antes de la regresión lasso
pipeline = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=0.90)), ('lasso', Lasso(alpha=0.1))])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


In [16]:
#Evaluando el modelo
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 29165.47
Coefficient of determination: 0.59


In [11]:
#viendo los coeficientes
df_coef = pd.DataFrame(pipeline.named_steps['lasso'].coef_, index=df.drop(columns='WhiteElo').columns, columns=['Coeficiente'])
df_coef

        Coeficiente
Event      3.552603
White     16.694406
Black      3.418207
Result    29.683704
Hour       1.168946
...             ...
586       -2.506392
587        0.438948
588        5.188515
589       -5.257790
590        0.315165

[600 rows x 1 columns]


## Regresión Ridge

In [20]:
# creando un pipeline antes de la regresión lasso
pipeline = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=1)), ('ridge', Ridge())])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [21]:
#Evaluando el modelo
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 29161.53
Coefficient of determination: 0.59


## Selección de características

In [5]:
import statsmodels.api as sm
from statsmodels.api import add_constant

In [6]:
# Separanado las variables dependientes e independientes
X = df.drop(columns='WhiteElo')
y = df['WhiteElo']  

X_ols = X.copy()
X_ols['intercept'] = 1

model = sm.OLS(y, X_ols)
results = model.fit()

# Asumiendo que X es un DataFrame de pandas
X_ols = add_constant(X, prepend=False)
model = sm.OLS(y, X_ols)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               WhiteElo   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.591
Method:                 Least Squares   F-statistic:                     2186.
Date:                Sun, 26 May 2024   Prob (F-statistic):               0.00
Time:                        23:28:16   Log-Likelihood:            -4.7002e+06
No. Observations:              716289   AIC:                         9.401e+06
Df Residuals:                  715814   BIC:                         9.407e+06
Df Model:                         474                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Event           2.0869      0.131     15.980      

In [7]:
# Obtener los p-values
pvalues = results.pvalues

# Filtrar las columnas de X_ols con p-values menores a 0.05
selected_columns = pvalues[pvalues < 0.05].index

print(selected_columns)

Index(['Event', 'White', 'Black', 'Result', 'Hour', 'BlackElo', 'Opening',
       'TimeControl', 'Termination', '194', '252', '266', '274', '298', '301',
       '315', '325', '338', '370', '400', '427', '436', '437', '438', '439',
       '445', '460', '464', '469', '473', '480', '482', '501', '504', '510',
       '514', '516', '519', '521', '525', '527', '528', '536', '542', '546',
       '548', '551', '554', '556', '557', '559', '561', '562', '564', '565',
       '567', '568', '569', '570', '571', '573', '574', '575', '576', '577',
       '579', '580', '582', '583', '585', '586', '587', '588', '589', 'const'],
      dtype='object')


In [8]:
selected_columns = selected_columns.drop('const')
X_selected = X[selected_columns]
X_selected

,Event,White,Black,Result,Hour,BlackElo,Opening,TimeControl,Termination,194,...,577,579,580,582,583,585,586,587,588,589
0,1,123,62,2,22,1570,28072,1074584,1,0,...,69,82,73,328,75,82,77,163,328,81
1,1,457,66,2,22,1436,595,1074584,1,0,...,95,60,104,462,110,311,116,739,169,128
2,0,44,216,2,22,1765,34752,1074584,1,0,...,275,181,297,183,320,181,350,166,172,376
3,2,35,1377,2,22,1673,11136,47338,1,0,...,450,77,101,81,105,84,101,333,88,105
4,4,44,187,2,22,1621,48705,105090,1,0,...,196,77,54,81,243,84,191,360,88,243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716284,0,121,108,1,16,1531,12341,1074584,1,0,...,265,18,368,19,753,20,433,675,21,1
716285,2,350,561,1,13,1807,7614,912414,1,0,...,11,164,12,1531,13,129,14,96,151,16
716286,5,352,50,1,19,1777,62971,579928,0,0,...,0,0,0,0,0,0,0,0,0,2
716287,1,227,161,1,6,1722,28072,948246,3,0,...,636,81,790,227,84,67,88,300,480,95


In [22]:
scaler = StandardScaler()
X_selected = scaler.fit_transform(X_selected)

# creando un pipeline antes de la regresión lasso
pipeline = Pipeline([('scaler', StandardScaler()), ('linealregression', LinearRegression())])
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [23]:
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 29126.76
Coefficient of determination: 0.59


In [ ]:
plt.plot(np.arange(0, len(y_test)), np.arange(0, len(y_test)))

# Conclusión
| Regresión | R2_score |
|-----------|----------|
|LinearRegression| 0.59 |
|Lasso|0.59|
|Ridge|0.59|
|LR Selected|0.59|
  
Por el valor de los coeficientes y el comportamiento de las gráficas, podemos concluir que el Elo del jugador con piezas blancas no tiene mucha relación con cómo se desempeño la partida. Esto puede deberse a que las condiciones de cada partida son muy diferentes y ni siquiera el puntaje Elo de los jugadores de las piezas negras puede explicar del todo la variable "objetivo", ya que, si bien existe una relación entre ambos puntajes, lo cierto es que el ajedrez es un juego complejo y muy diverso, por lo que pueden darse partidas con una gran diferencia de elo entre jugadores. Una posible razón para esto, es que entre mayor sea la diferencia de puntajes, más grande es el aumento del elo para el jugador con el puntaje más bajo.  
Hablando en términos estadísticos, el elo no es una consecuencia de las demás variables, es decir, no hay una causalidad. Por ejemplo, no por que ganaron las piezas blancas y el elo de las piezas blancas era bajo, significa que eso produce que el elo del jugador con las piezas blancas se vuelva blanco, es decir, esto no es la consecuencia de las demás variables. Una de las hipótesis de la regresión lineal, es la causalidad de variable objetivo, pero este no es el caso. Esta regresión se realizo únicamente con fines didácticos y es de hecho un buen ejercicio el notar este tipo de hipótesis intrínsecas que conlleva la regresión lineal.  
En el notebook de clasificación se intentará predecir la variable "Result" de este mismo dataset, la cuál nos dice el resultado de la partida. Esta variable si demuestra la causalidad de las demás variables de este dataset, por lo que se esperan mejores resultados.